# EU Proceedings - Danish

In [1]:
import glob
import regex
import numpy as np
from collections import Counter

from sklearn.model_selection import train_test_split
from keras import regularizers
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, LSTM, Conv1D, MaxPooling1D

Using TensorFlow backend.


In [2]:
import komma_dev.eu
from komma_dev.vocabulary import build_vocabulary
from komma_dev.parsing import StringParser

In [3]:
eu_data_raw = komma_dev.eu.load('da')

loading: europarl-v7.da-en.da


We remove leading and trailing whitespace. When running on real user input this should be done separately and the amount and kind of removed whitespace should be stored and eventually added back onto the string.

In [4]:
eu_data_raw = [line.strip() for line in eu_data_raw]

We now parse each string using our StringParser class. This may take a while.

In [5]:
parser = StringParser()
eu_data = [parser.parse(line) for line in eu_data_raw]

## Split
The very first thing we need to do after having loaded the data is to create a train/dev split. We will eventually build a vocabulary, and it's important that we build this on the training data only.

In [7]:
eu_data_train, eu_data_dev = train_test_split(eu_data, test_size=0.05, random_state=42)

In [8]:
len(eu_data_train)

1870360

In [9]:
len(eu_data_dev)

98440

# Vocabulary
We can now build the vocabulary on the training data.

In [10]:
vocab, _ = build_vocabulary(eu_data_train, 50000)

In [11]:
len(vocab)

50000

In [12]:
MAX_CHUNKS = 50

def embed_sentence(sentence, vocabulary):
    embedded = [0] * MAX_CHUNKSMAX_TOKENS
    for index, chunk in enumerate(sentence.chunks[:MAX_CHUNKSMAX_TOKENS]):
        word = chunk.clean_name
        if word in vocabulary:
            word_id = vocabulary[word]
            embedded[index] = word_id
        else:
            # TODO: don't encode unknown as zero (probably use max(IDs) + 1)
            embedded[index] = 0
    return embedded

def pre_process(sentences, vocabulary):
    X = []
    Y = []
    for sentence in sentences:
        # TODO: Remove punctuation except comma?
        # sentence = remove_punctuation(sentence)
        y = sentence.commas[:50] + ([0] * (MAX_CHUNKS-len(sentence.commas)))
        y = np.array(y)[np.newaxis]
        embedded_sentence = embed_sentence(sentence, vocabulary)
        X.append(embedded_sentence)
        Y.append(y)
    return X, Y

In [13]:
X_train, Y_train = pre_process(eu_data_train, vocab)

In [14]:
print(len(X_train))
print(len(Y_train))

1870360
1870360


In [15]:
X_dev, Y_dev = pre_process(eu_data_dev, vocab)

In [16]:
print(len(X_dev))
print(len(Y_dev))

98440
98440


In [17]:
# TODO: This only called to get NP arrays, so maybe make sure the X and Y are already NP arrays.
X_train = sequence.pad_sequences(X_train, maxlen=50)
Y_train = np.concatenate(Y_train, axis=0)

X_dev = sequence.pad_sequences(X_dev, maxlen=50)
Y_dev = np.concatenate(Y_dev, axis=0)

In [18]:
print('X_train shape:', X_train.shape)
print('Y_train shape:', Y_train.shape)
print('X_dev shape:', X_dev.shape)
print('Y_dev shape:', Y_dev.shape)

X_train shape: (1870360, 50)
Y_train shape: (1870360, 50)
X_dev shape: (98440, 50)
Y_dev shape: (98440, 50)


# Network

In [19]:
model = Sequential()
model.add(Embedding(len(vocab) + 1, 128, input_length=50))
model.add(Dropout(0.5))
model.add(Conv1D(filters=64,
                 kernel_size=5,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(50))
model.add(Dense(50))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [20]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 128)           6400128   
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 128)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 46, 64)            41024     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 11, 64)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                23000     
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
activation_1 (Activation)    (None, 50)                0         
Total para

# Train

In [21]:
history = model.fit(X_train, Y_train,
          batch_size=128,
          epochs=5,
          verbose=1,
          validation_data=(X_dev, Y_dev))

Train on 1870360 samples, validate on 98440 samples
Epoch 1/5
1870360/1870360 [==============================] - 2415s - loss: 0.0724 - acc: 0.9772 - val_loss: 0.0428 - val_acc: 0.9871
Epoch 2/5
1870360/1870360 [==============================] - 2576s - loss: 0.0386 - acc: 0.9885 - val_loss: 0.0321 - val_acc: 0.9907
Epoch 3/5
1870360/1870360 [==============================] - 2582s - loss: 0.0320 - acc: 0.9906 - val_loss: 0.0283 - val_acc: 0.9918
Epoch 4/5
1870360/1870360 [==============================] - 2457s - loss: 0.0293 - acc: 0.9913 - val_loss: 0.0268 - val_acc: 0.9922
Epoch 5/5
1870360/1870360 [==============================] - 2573s - loss: 0.0279 - acc: 0.9917 - val_loss: 0.0260 - val_acc: 0.9924


# Prediction

In [69]:
def apply_commas(sentence, commas):
    assert len(sentence.chunks) == len(commas)
    temp = ""
    for chunk, comma in zip(sentence.chunks, commas):
        if comma == chunk.comma:
            temp += chunk.name + chunk.trailing_whitespace
            continue
        
        if comma:
            temp += chunk.name + "," + chunk.trailing_whitespace
        else:
            temp += chunk.name[:-1] + chunk.trailing_whitespace
    return temp

In [87]:
#example = 'Dette er en sætning som der burde indholde et komma .'
#example = """det bliver bedre hvis du øver dig lidt først"""
#example = "jeg spiser kage hvad spiser du?"
#example = "ved du hvad tid toget kører?"
#example = str(eu_data_dev[1])
#example = "Og til gengæld sker der det og jeg gentager at den samlede landbrugsstøtte vil stige med hele 8% indtil 2013 uden at medregne optagelsen af Bulgarien og Rumænien der er planlagt til 2007."
example = "Ja det var hyggeligt."
example = parser.parse(example)
embedded = embed_sentence(example, vocab)
embedded = sequence.pad_sequences([embedded], maxlen=50)
yhat = model.predict_proba(embedded)
y_hat_commas = yhat[0][:len(example.chunks)] >= 0.5
print("original:", example)
print("fixed:", apply_commas(example, y_hat_commas))


1/1 [==============================] - 0s
original: Ja det var hyggeligt.
fixed: Ja, det var hyggeligt.
